In [1]:
# Import system modules
import sys, os
sys.path.insert(0, os.path.abspath('..'))

# Import project modules
from src.map_viewer_v4 import MapViewer
from src.search_algorithm_v1 import DroneSearchSimulation
from src.geocode_v4 import geocode_place
from src.area_calc_v4 import compute_area
from src.config_v4 import (BHUVAN_LAYERS, DEFAULT_CENTER, DEFAULT_ZOOM, 
                         DEFAULT_BASE_LAYER, DEFAULT_OVERLAYS, DEFAULT_DRONES,
                         MAX_SIMULATION_STEPS, DRONE_COLORS)

# Import utility modules
from src.utils.map_utils import update_map_layers
from src.utils.simulation_utils import run_simulation, display_coverage_progression
from src.utils.ui_utils import create_map_ui, create_search_ui, create_layout

# UI components
from IPython.display import display, clear_output, HTML

# ---- Initialize map viewer ----
mv = MapViewer(center=DEFAULT_CENTER, zoom=DEFAULT_ZOOM)
m = mv.get_map()

# ---- State variables ----
active_base = DEFAULT_BASE_LAYER
active_overlays = set(DEFAULT_OVERLAYS)
current_polygon_coords = None
simulation = None
is_simulating = False

# ---- Create UI components ----
map_ui = create_map_ui(BHUVAN_LAYERS, DEFAULT_BASE_LAYER)
search_ui = create_search_ui(DEFAULT_DRONES)
layout = create_layout(map_ui, search_ui)

# ---- Event handlers ----

# Base layer change handler
def on_base_change(change):
    global active_base
    if change['name'] == 'value':
        active_base = change['new']
        with search_ui['status_out']:
            clear_output()
            print(f"Loading base layer: {active_base}...")
        update_map_layers(mv, active_base, active_overlays, BHUVAN_LAYERS, 
                         lambda msg: print(msg, file=search_ui['status_out']))

map_ui['base_selector'].observe(on_base_change, names='value')

# Overlay checkbox handlers
def create_overlay_handler(layer_name):
    def handler(change):
        if change['name'] == 'value':
            is_active = change['new']
            with search_ui['status_out']:
                clear_output()
                if is_active:
                    active_overlays.add(layer_name)
                    print(f"Adding overlay: {layer_name}")
                else:
                    active_overlays.discard(layer_name)
                    print(f"Removing overlay: {layer_name}")
            update_map_layers(mv, active_base, active_overlays, BHUVAN_LAYERS, 
                             lambda msg: print(msg, file=search_ui['status_out']))
    return handler

for layer_name, checkbox in map_ui['overlay_checkboxes'].items():
    checkbox.observe(create_overlay_handler(layer_name), names='value')

# Search handler
def on_search(b):
    with search_ui['area_out']:
        clear_output()
        with search_ui['status_out']:
            clear_output()
            print(f"Searching for: {map_ui['search_box'].value}...")
        try:
            lat, lon = geocode_place(map_ui['search_box'].value)
            m.center = (lat, lon)
            m.zoom = 12
            with search_ui['status_out']:
                clear_output()
                print(f"Found location: {map_ui['search_box'].value} ({lat}, {lon})")
        except Exception as e:
            with search_ui['status_out']:
                clear_output()
                print(f"Geocoding error: {e}")

map_ui['search_btn'].on_click(on_search)

# Draw handler for area calculation and enabling search
def handle_draw(target, action, geo_json):
    global current_polygon_coords, simulation
    
    # Reset simulation if running
    if is_simulating:
        stop_simulation(None)
    
    # Extract polygon coordinates from geo_json
    if geo_json['geometry']['type'] == 'Polygon':
        coords = geo_json['geometry']['coordinates'][0]
        # Convert from [lon, lat] to (lat, lon) for leaflet
        current_polygon_coords = [(lat, lon) for lon, lat in coords]
        
        # Calculate area
        area = compute_area(coords, map_ui['unit_selector'].value)
        with search_ui['area_out']:
            clear_output()
            print(f"Area: {area:.3f} {map_ui['unit_selector'].label}")
        
        # Enable start search button
        search_ui['start_search_btn'].disabled = False
        
        # Create new simulation instance
        simulation = DroneSearchSimulation(
            area_coords=[(lon, lat) for lon, lat in current_polygon_coords],
            n_drones=search_ui['drone_count'].value
        )
        
        with search_ui['status_out']:
            clear_output()
            print(f"Ready to start search with {search_ui['drone_count'].value} drones.")

mv.on_draw(handle_draw)

# Handler for drone count change
def on_drone_count_change(change):
    global simulation
    if change['name'] == 'value' and current_polygon_coords is not None:
        # Re-initialize simulation with new drone count
        simulation = DroneSearchSimulation(
            area_coords=[(lon, lat) for lon, lat in current_polygon_coords],
            n_drones=change['new']
        )
        with search_ui['status_out']:
            clear_output()
            print(f"Updated to {change['new']} drones for simulation.")

search_ui['drone_count'].observe(on_drone_count_change, names='value')

# Start simulation handler
def start_simulation(b):
    global is_simulating
    
    try:
        if simulation is None or current_polygon_coords is None:
            with search_ui['status_out']:
                clear_output()
                print("Error: No search area defined. Please draw a polygon first.")
            return
        
        # Set simulation state
        is_simulating = True
        search_ui['start_search_btn'].disabled = True
        search_ui['stop_search_btn'].disabled = False
        
        # Show the coverage progress bar
        search_ui['coverage_progress'].layout.visibility = 'visible'
        
        # Clear existing simulation layers
        mv.clear_simulation_layers()
        
        # Clear the simulation output
        with search_ui['simulation_out']:
            clear_output()
            print("Starting search simulation...")
        
        # Run the simulation
        simulation_steps = [0, 24, 49, 74, 99]  # Steps to save visualizations at
        
        # Progress update function
        def update_progress(value):
            search_ui['coverage_progress'].value = value
        
        # Status update function
        def update_status(message):
            with search_ui['simulation_out']:
                print(message)
        
        # Run simulation and get results
        results = run_simulation(
            simulation, 
            MAX_SIMULATION_STEPS, 
            simulation_steps,
            update_progress,
            update_status
        )
        
        # Display coverage progression
        with search_ui['simulation_out']:
            clear_output()
            print(f"Simulation complete!")
            print(f"Final coverage: {results['final_result']['coverage_percent']:.1f}%")
            display(display_coverage_progression(results['visualizations']))
        
        # Add drone visualization to the map
        for i, (pos, path) in enumerate(zip(results['drone_positions'], results['drone_paths'])):
            # Use color from the config
            color = DRONE_COLORS[i % len(DRONE_COLORS)]
            
            # Add drone marker (lat, lon order for leaflet)
            mv.add_drone_marker(
                position=(pos[1], pos[0]),
                color=color,
                radius=5
            )
            
            # Add drone path if it has at least 2 points
            if len(path) >= 2:
                # Convert coordinates for leaflet (lat, lon)
                path_coords = [(p[1], p[0]) for p in path]
                mv.add_drone_path(
                    path=path_coords,
                    color=color,
                    weight=2,
                    opacity=0.7
                )
        
        # Reset simulation state
        is_simulating = False
        search_ui['start_search_btn'].disabled = False
        search_ui['stop_search_btn'].disabled = True
        
    except Exception as e:
        # Handle errors
        with search_ui['simulation_out']:
            clear_output()
            import traceback
            print(f"Error in simulation: {str(e)}")
            print("\nFull error details:")
            print(traceback.format_exc())
            
        # Reset simulation state
        is_simulating = False
        search_ui['start_search_btn'].disabled = False
        search_ui['stop_search_btn'].disabled = True

search_ui['start_search_btn'].on_click(start_simulation)

# Stop simulation handler
def stop_simulation(b):
    global is_simulating
    
    is_simulating = False
    search_ui['start_search_btn'].disabled = False
    search_ui['stop_search_btn'].disabled = True
    
    with search_ui['simulation_out']:
        clear_output()
        print("Simulation stopped.")

search_ui['stop_search_btn'].on_click(stop_simulation)

# ---- Display UI ----
display(HTML("<h3>Bhuvan Map Viewer v4 with Search Simulation</h3>"))
display(layout['search_controls'])
display(layout['main_tabs'])
display(m)

# Initialize with default base layer
with search_ui['status_out']:
    print(f"Initializing map with {active_base}...")
update_map_layers(mv, active_base, active_overlays, BHUVAN_LAYERS, 
                 lambda msg: print(msg, file=search_ui['status_out']))

ImportError: cannot import name 'BUHVAN_GEOCODE_URL' from 'src.config_v4' (C:\Users\garg2\Documents\AasmaSAR_Maps\src\config_v4.py)